# Dataset anonymization

##### Import lib for data-working

In [1]:
import pandas as pd

##### Read input file

In [2]:
filename = 'dataset259999.xlsx' # <- filename
df = pd.read_excel(filename)
df

,ФИО работника,Номер телефона,Адрес работы,Должность,Заработная плата
0,Ильюшкин Салман Айратович,79312401293,ул. Чайковского д. 61 стр А,Генеральный директор,84800
1,Бровиков Хасан Валерьянович,79030995018,пр-кт Владимирский д. 10 стр А,Генеральный директор,86400
2,Федчина Ия Рашадовна,79817186622,ул. Чайковского д. 81 стр А,Генеральный директор,81600
3,Кондраценка Лавс Октавович,79817317562,Павлоградский пер. д. 3а стр 1,Генеральный директор,81600
4,Просвиркина Настасья Аланусовна,79043309894,ул. Марата д. 92,Генеральный директор,86400
...,...,...,...,...,...
259994,Цыбасов Якун Авлович,79110050095,пр-кт Большой В.О. д. 67 стр А,Менеджер,53500
259995,Онопченко Никифор Всеволодович,79213525928,ул. Константина Заслонова д. 22 стр А,Инженер,30900
259996,Сысоев Тадеуш Терентиевич,79312334226,Павлоградский пер. д. 1 стр А,Финансовый директор,98000
259997,Курзаков Аристарх Тарасович,79118108304,пр-кт Загородный д. 42 стр В,Менеджер,45000


##### Flags

In [3]:
ANON_JOBS = True; ANON_ADDRESS = True; ANON_SALARY = True
LOCAL_SUPRESS = False; HIDE_ADDRESS = False; SAVE_ANON_FILE = False

if LOCAL_SUPRESS:
    REQUIRED_K_ANON = 5

##### Anonymization funcs

In [4]:
# Salary anonymization
def salary_local_generalization(x):
    val = int(x)
    k10 = val // 10000
    return str(k10 * 10000) + '-' + str((k10 + 1) * 10000)

# Name anonymize
def del_attrib(x):
    return '***'

# Address anonymize
vas_distr = ["5-я В.О. линия",
"8-я В.О. линия",
"11-я В.О. линия",
"ул. Камская",
"9-я В.О. линия",
"14-я В.О. линия",
"пр-кт Средний В.О."
"4-я В.О. линия",
"12-я В.О. линия",
"2-я В.О. линия",
"16-я В.О. линия",
"3-я В.О. линия",
"17-я В.О. линия",
"10-я В.О. линия",
"18-я В.О. линия",
"15-я В.О. линия",
"19-я В.О. линия",
"Реки Смоленки наб."
"ул. Донская",
"пр-кт Большой В.О.",
"7-я В.О. линия",
"13-я В.О. линия",
"Макарова наб.",
"6-я В.О. линия",
"пр-кт Малый В.О."]
def addr_local_generalization(x):
    street = str(x.split(" д. ")[0])
    if street in vas_distr:
        return 'Василеостровский'
    else:
        return 'Центральный'

# Job anonymize
def job_local_generalization(x):
    s = str(x)
    if x == "Административный директор" or x == "Директор по маркетингу" or x == "Финансовый директор" or x == "Генеральный директор":
        return "Директор"
    elif x == "Водитель" or x == "Комендант" or x == "Охранник" or x == "Уборщик" or x == "Секретарь" or x == "Бухгалтер":
        return "Вспомогательный персонал"
    else:
        return s

# Phone anonymize
def phone_local_generalization(x):
    s = str(x)
    if int(s[1:4]) in [929, 921, 931]:
        return "Megafon"
    elif int(s[1:4]) in [911, 981]:
        return "MTS"
    elif int(s[1:4]) in [961, 962, 963, 964, 903, 905, 906, 909, 960]:
        return "Beeline"
    elif int(s[1:4]) in [901, 952, 904, 950, 951]:
        return "Tele2"

if ANON_SALARY:
    df["Заработная плата"] = df["Заработная плата"].apply(salary_local_generalization)

# Direct identifier - always anonymize
df["ФИО работника"] = df["ФИО работника"].apply(del_attrib)


if ANON_ADDRESS:
    df["Адрес работы"] = df["Адрес работы"].apply(addr_local_generalization)

if ANON_JOBS:
    df["Должность"] = df["Должность"].apply(job_local_generalization)

if HIDE_ADDRESS:
    df["Адрес работы"] = df["Адрес работы"].apply(del_attrib)
# Direct identifier - always anonymize
df["Номер телефона"] = df["Номер телефона"].apply(phone_local_generalization)

#### Get dictionary [field : frequency] and k-anonymity

In [5]:
def get_dict_and_k_anonimity():
    # my_dictionary is used to count k-anonymity
    my_dictionary = {}
    # key is concatenated row
    def get_row_as_string(ind):
        return str(df["ФИО работника"].iloc[ind]) +\
               str(df["Номер телефона"].iloc[ind]) +\
               str(df["Адрес работы"].iloc[ind]) +\
               str(df["Должность"].iloc[ind]) +\
               str(df["Заработная плата"].iloc[ind])
    for row in range(df.shape[0]):
        s = get_row_as_string(row)
        if s in my_dictionary:
            my_dictionary[s] += 1
        else:
            my_dictionary[s] = 1
    return my_dictionary, min(my_dictionary.values())

##### List of strings of each row

In [6]:
# return list of rows as single string
strings = [''.join(val) for val in df.astype(str).values.tolist()]

#### Output

In [7]:
mydict, k_anon = get_dict_and_k_anonimity()

print("k-anonymity = ", k_anon)

# sorted dictionary = list of tuples as [(key, value) for key,value in dict]
sorted_mydict = sorted(mydict.items(), key=lambda x:x[1])
print("Unique rows:", len(mydict))
print("5 least frequent rows:")

# line below extracts from dataframe 5 least frequent lines (because the corresponding keys are strings) and display them as dataframe
display(df.loc[[strings.index(sorted_mydict[i][0]) for i in range(5)]])
print("Frequencies : ", [sorted_mydict[i][1] for i in range(5)])

k-anonymity =  11
Unique rows: 328
5 least frequent rows:


,ФИО работника,Номер телефона,Адрес работы,Должность,Заработная плата
89478,***,Tele2,Василеостровский,Программист,110000-120000
679,***,Beeline,Василеостровский,Программист,110000-120000
14979,***,MTS,Василеостровский,Программист,110000-120000
34929,***,Beeline,Центральный,Программист,110000-120000
18045,***,Tele2,Василеостровский,Директор,110000-120000


Frequencies :  [11, 17, 22, 26, 38]


##### Local suppress

In [8]:
if LOCAL_SUPRESS:
    forDelete = []

    for index in range(df.shape[0]):
        line = strings[index]
        if mydict[line] < REQUIRED_K_ANON:
            #print(line, ">>", index)
            forDelete.append(index)
    print(len(forDelete), " lines need to be suppressed to meet all requirements")

#### Output after local suppress

In [9]:
if LOCAL_SUPRESS:
    df = df.drop(df.index[forDelete])
    new_dict, new_k_anon = get_dict_and_k_anonimity()
    print("New k-anon = ", new_k_anon)
    print("New unique rows:", len(new_dict))
    sorted_new_dict = sorted(new_dict.items(), key=lambda x:x[1])
    strings = [''.join(val) for val in df.astype(str).values.tolist()]
    print("New 5 least frequent rows:")
    display(df.loc[[strings.index(sorted_new_dict[i][0]) for i in range(5)]])
    print("Frequencies : ", [sorted_new_dict[i][1] for i in range(5)])

##### AVG k-anonymity calculation

In [10]:
sum_of_frequencies = 0
def get_dict():
    if LOCAL_SUPRESS:
        return new_dict
    else:
        return mydict
for key in get_dict():
    sum_of_frequencies += get_dict()[key]
print("AVG k-anonymity = ", round(sum_of_frequencies / len(get_dict()), 2))

AVG k-anonymity =  792.68


### Anonymized dataset

In [11]:
df

,ФИО работника,Номер телефона,Адрес работы,Должность,Заработная плата
0,***,Megafon,Центральный,Директор,80000-90000
1,***,Beeline,Центральный,Директор,80000-90000
2,***,MTS,Центральный,Директор,80000-90000
3,***,MTS,Центральный,Директор,80000-90000
4,***,Tele2,Центральный,Директор,80000-90000
...,...,...,...,...,...
259994,***,MTS,Василеостровский,Менеджер,50000-60000
259995,***,Megafon,Центральный,Инженер,30000-40000
259996,***,Megafon,Центральный,Директор,90000-100000
259997,***,MTS,Центральный,Менеджер,40000-50000


#### Save anonymized dataset as .xlsx

In [12]:
path = filename.split('.')[0] + 'anonymized.xlsx'

if SAVE_ANON_FILE:
    df.to_excel(path, index = False)